In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from fuzzywuzzy import fuzz, process

# Importieren der Excel-Datei (sie liegt im gleichen Ordner wie dieses Notebook)
geburtstag = "Geburstage_2026.xlsx"

df = pd.read_excel(geburtstag, engine="openpyxl")


In [16]:
df.head(20)

,Kontakte,Firma,Anredeart,Vorname,Nachname,Strasse (Korr.),PLZ (Korr.),Ort (Korr.),Geburtsdatum,Korresp.sprache,Mitgliedschaft
0,"Adaastra Boutique Hotel (Imhof, Christian Frank)",Adaastra Boutique Hotel,Du-Form,Christian Frank,Imhof,Furkastrasse 16,3904,Naters,06.08.1970,Deutsch,Aktiv
1,"Adler Gersau GmbH (Schadowske, Volker)",Adler Gersau GmbH,Du-Form,Volker,Schadowske,Dorfstrasse 11,6442,Gersau,06.05.1970,Deutsch,Aktiv
2,"Aeschlimann, Hans-Jürg",NaN,Sie-Form,Hans-Jürg,Aeschlimann,Oberi Chros 18,2513,Twann,20.01.1949,Deutsch,Passiv
3,"Albergo Croce Bianca (Zanolari, Claudio)",Albergo Croce Bianca,Du-Form,Claudio,Zanolari,Via da Mez 97\nCasella postale 135,7742,Poschiavo,28.03.1974,Deutsch,Aktiv
4,"ANICO GmbH (Schmid, Nicole)",ANICO GmbH,Du-Form,Nicole,Schmid,Untere Steig 15,8462,Rheinau,18.12.1993,Deutsch,Aktiv
5,"Antico Grotto Ticino (Raith, Odette)",Antico Grotto Ticino,Sie-Form,Odette,Raith,Via Guiccio 15,6834,Morbio Inferiore,30.08.1956,Deutsch,Antrag Aktiv-Passiv
6,"Auberge Les Viviers (Flury, Valentin)",Auberge Les Viviers,Du-Form,Valentin,Flury,Route de Develier 24,2800,Delémont,17.02.1963,Französisch,Aktiv
7,"AZ Gastro AG Restaurant Burestube (Camenzind, ...",AZ Gastro AG Restaurant Burestube,Du-Form,Rita,Camenzind,Bachstrasse 3,5033,Buchs,19.03.1975,Deutsch,Aktiv
8,"AZ Gastronomie AG (Zaugg, Andy)",AZ Gastronomie AG,Sie-Form,Andy,Zaugg,Friedhofplatz 10,4500,Solothurn,21.06.1966,Deutsch,Passiv
9,"B&B Rose Kerns GmbH (della Torre, Bruno)",B&B Rose Kerns GmbH,Du-Form,Bruno,della Torre,Dorfstrasse 5,6064,Kerns,11.09.1980,Deutsch,Aktiv


In [17]:
# Die Spalte 'Geburtstag' wird in das Datetime-Format konvertiert (Monat/Tag ignoriert Jahr, wenn angegeben)
df['Geburtsdatum'] = pd.to_datetime(df['Geburtsdatum'], errors='coerce', dayfirst=True)

# Hilfsspalte für Sortierung nach Monat und Tag
df['_geburtstag_sort'] = df['Geburtsdatum'].dt.strftime('%m-%d')

# Sortiere DataFrame vom 01.01. bis 31.12.
df = df.sort_values('_geburtstag_sort').reset_index(drop=True)

# Hilfsspalte wieder entfernen
df = df.drop('_geburtstag_sort', axis=1)


In [18]:
df.head(20)

,Kontakte,Firma,Anredeart,Vorname,Nachname,Strasse (Korr.),PLZ (Korr.),Ort (Korr.),Geburtsdatum,Korresp.sprache,Mitgliedschaft
0,"Werner Rätz (Rätz, Werner)",Werner Rätz,Du-Form,Werner,Rätz,Moosgasse 5,3215,Büchslen,1959-01-01,Deutsch,Passiv
1,"Chicherio, Remo",NaN,Sie-Form,Remo,Chicherio,Schlüsselmattweg 12,8840,Einsiedeln,1979-01-01,Deutsch,Passiv
2,"Kaufmann, René",NaN,Du-Form,René,Kaufmann,Neubrunn 823,8488,Turbenthal,1962-01-02,Deutsch,Ehrenmitglied
3,"Landgasthof Seelust AG (Hasen, Mattias)",Landgasthof Seelust AG,Du-Form,Mattias,Hasen,Wiedehorn,9322,Egnach,1984-01-03,Deutsch,Aktiv
4,"Riedener, Thomas",NaN,Sie-Form,Thomas,Riedener,Silserweg 4,7430,Thusis,1965-01-03,Deutsch,Passiv Austritt
5,"Hotel Waldsägmühle GmbH &Co.KG (Ziegler, Martin)",Hotel Waldsägmühle GmbH &Co.KG,Du-Form,Martin,Ziegler,Waldsägemühle 1,D-72285,Pfalzgrafenweiler-Kälberbronn,1970-01-04,Deutsch,Aktiv
6,"Restaurant Bahnhöfli Entlebuch AG (Felder, Willi)",Restaurant Bahnhöfli Entlebuch AG,Du-Form,Willi,Felder,Bahnhofstrasse 30,6162,Entlebuch,1986-01-05,Deutsch,Aktiv
7,"Birchmeier, Christian",NaN,Du-Form,Christian,Birchmeier,Fanenrietstrasse 1\nHaus A,7320,Sargans,1969-01-06,Deutsch,Ehrenmitglied
8,"Hotel al ponte (Walker, Frank)",Hotel al ponte,Du-Form,Frank,Walker,Wangenstrasse 55,3380,Wangen an der Aare,1972-01-07,Deutsch,Aktiv
9,"Hotel Frohe Aussicht (Inauen, Arno)",Hotel Frohe Aussicht,Du-Form,Arno,Inauen,Küchenrain 11,9057,Schwende,1975-01-07,Deutsch,Aktiv


In [19]:
df.drop(columns=['Kontakte', 'Anredeart'], inplace=True)

# Benenne die Spalten 'Strasse (Korr.)', 'PLZ (Korr.)', 'Ort (Korr.)' um (entferne ' (Korr.)')
df = df.rename(columns={
    'Strasse (Korr.)': 'Strasse',
    'PLZ (Korr.)': 'PLZ',
    'Ort (Korr.)': 'Ort'
})

df.head(20)

,Firma,Vorname,Nachname,Strasse,PLZ,Ort,Geburtsdatum,Korresp.sprache,Mitgliedschaft
0,Werner Rätz,Werner,Rätz,Moosgasse 5,3215,Büchslen,1959-01-01,Deutsch,Passiv
1,NaN,Remo,Chicherio,Schlüsselmattweg 12,8840,Einsiedeln,1979-01-01,Deutsch,Passiv
2,NaN,René,Kaufmann,Neubrunn 823,8488,Turbenthal,1962-01-02,Deutsch,Ehrenmitglied
3,Landgasthof Seelust AG,Mattias,Hasen,Wiedehorn,9322,Egnach,1984-01-03,Deutsch,Aktiv
4,NaN,Thomas,Riedener,Silserweg 4,7430,Thusis,1965-01-03,Deutsch,Passiv Austritt
5,Hotel Waldsägmühle GmbH &Co.KG,Martin,Ziegler,Waldsägemühle 1,D-72285,Pfalzgrafenweiler-Kälberbronn,1970-01-04,Deutsch,Aktiv
6,Restaurant Bahnhöfli Entlebuch AG,Willi,Felder,Bahnhofstrasse 30,6162,Entlebuch,1986-01-05,Deutsch,Aktiv
7,NaN,Christian,Birchmeier,Fanenrietstrasse 1\nHaus A,7320,Sargans,1969-01-06,Deutsch,Ehrenmitglied
8,Hotel al ponte,Frank,Walker,Wangenstrasse 55,3380,Wangen an der Aare,1972-01-07,Deutsch,Aktiv
9,Hotel Frohe Aussicht,Arno,Inauen,Küchenrain 11,9057,Schwende,1975-01-07,Deutsch,Aktiv


In [20]:
# Füge eine neue Spalte 'Alter 2026' hinzu, die berechnet, wie alt die Person an ihrem Geburtstag im Jahr 2026 wird
df['Alter 2026'] = 2026 - df['Geburtsdatum'].dt.year

df.sample(20)

,Firma,Vorname,Nachname,Strasse,PLZ,Ort,Geburtsdatum,Korresp.sprache,Mitgliedschaft,Alter 2026
207,Restaurant Stiva Veglia,Tino,Zimmermann,Via Vitg 51,7130,Schnaus,1975-08-21,Deutsch,Aktiv,51.0
289,NaN,Christian,Jurczyk,Via Maistra 128,7505,Celerina-St. Moritz,1949-11-07,Deutsch,Passiv,77.0
202,Gasthof Adler Sempach,Hanspeter,Künzli,Stadtstrasse 22,6204,Sempach,1963-08-19,Deutsch,Aktiv,63.0
196,NaN,Dieter,Roth,Badhübel 348,5728,Gontenschwil,1947-08-11,Deutsch,Passiv,79.0
267,Marco Putschert,Marco,Purtschert,Beim Kreuz 7,4800,Langnau bei Reiden,1969-10-18,Deutsch,Passiv,57.0
162,Panorama Genussmanufaktur,Daniel,Rindisbacher,Aeschiriedstrasse 36,3703,Aeschi ob Spiez/Aeschiried,1961-06-21,Deutsch,Ehrenmitglied,65.0
303,Thurberg AG,Daniel,Franz,Thurbergstrasse 29,8570,Weinfelden,1958-11-20,Deutsch,Aktiv,68.0
340,NaN,Rolf,Schumacher,Rüfistrasse 6,7076,Parpan,1955-12-25,Deutsch,Passiv,71.0
137,Gasthaus zum Kantonsschild,Fritz,Leicht-Flühmann,Hauptstrasse 24,3215,Gempenach,1960-05-25,Deutsch,Aktiv,66.0
128,NaN,Josef,Pelzmann,Burgweg 3,3294,Büren an der Aare,1945-05-18,Deutsch,Passiv,81.0


In [ ]:
import pandas as pd
import numpy as np

# ExcelWriter erstellen
with pd.ExcelWriter('Geburtstagsliste_2026.xlsx', engine='xlsxwriter') as writer:
    # Erstelle zusätzliche Spalten falls noch nicht vorhanden
    df['Monat'] = df['Geburtsdatum'].dt.month
    df['Tag'] = df['Geburtsdatum'].dt.day

    # Name des Monats für das Tabellensheet (in Deutsch)
    monat_namen = {
        1: 'Januar', 2: 'Februar', 3: 'März', 4: 'April',
        5: 'Mai', 6: 'Juni', 7: 'Juli', 8: 'August',
        9: 'September', 10: 'Oktober', 11: 'November', 12: 'Dezember'
    }

    # Aufteilung nach Monaten
    for monat in range(1, 13):
        df_monat = df[df['Monat'] == monat].copy()
        if df_monat.empty:
            continue

        # Sortiere nach Tag
        df_monat = df_monat.sort_values('Tag')

        # Anzeigeformat für Geburtsdatum als 'dd.mm.yyyy'
        df_monat['Geburtsdatum (TT.MM.JJJJ)'] = df_monat['Geburtsdatum'].dt.strftime('%d.%m.%Y')

        # Füge Spalte 'Mitgliedschaft' hinzu (ggf. vorhandene Spalte nutzen)
        mitgliedschaft_col = None
        for mgl_col in ['Mitgliedschaft', 'mitgliedschaft', 'Mitglied', 'mitglied']:
            if mgl_col in df_monat.columns:
                mitgliedschaft_col = mgl_col
                break

        # Spaltenauswahl: Vorname, Nachname, Geburtsdatum (TT.MM.JJJJ), Firma, Strasse, PLZ, Ort, Alter 2026, Mitgliedschaft
        # Die Originalspalte "Geburtsdatum" NICHT übernehmen
        desired_cols = [
            'Vorname', 'Nachname', 'Geburtsdatum (TT.MM.JJJJ)',
            'Firma', 'Strasse', 'PLZ', 'Ort', 'Alter 2026'
        ]
        if mitgliedschaft_col:
            desired_cols.append(mitgliedschaft_col)
        # Nur existierende Spalten behalten
        sheet_cols = [c for c in desired_cols if c in df_monat.columns]
        export_df = df_monat[sheet_cols]

        # Schreibe DataFrame in Excel-Sheet
        sheet_name = monat_namen.get(monat, f"Monat_{monat}")
        export_df.to_excel(writer, sheet_name=sheet_name, index=False)

        # Zugriff aufs Sheet für Formatierungen
        workbook  = writer.book
        worksheet = writer.sheets[sheet_name]

        # Excel Formate definieren
        alter_format = workbook.add_format({'num_format': '0'})
        date_format_german = workbook.add_format({'num_format': 'dd.mm.yyyy'})
        green_highlight_format = workbook.add_format({'bg_color': '#C6EFCE', 'font_color': '#006100'}) # Excel Grün
        # Spezielles Hervorhebungsformat für Ehrenmitglied etc.
        yellow_highlight_format = workbook.add_format({'bg_color': '#FFF2CC', 'font_color': '#7F6000'})
        # Fettdruck für Header
        bold_format = workbook.add_format({'bold': True})
        worksheet.set_row(0, None, bold_format)
        # Spaltenbreiten setzen (Alle Spalten etwas schön machen)
        worksheet.set_column(0, len(sheet_cols)-1, 18)

        # Runde Geburtstage grün hervorheben
        if 'Alter 2026' in sheet_cols:
            alter_col_idx = sheet_cols.index('Alter 2026')
            col_letter = chr(ord('A') + alter_col_idx)
            row_end = len(export_df)
            rng = f'{col_letter}2:{col_letter}{row_end+1}'
            worksheet.conditional_format(
                rng,
                {
                    'type':     'formula',
                    'criteria': f'=AND(MOD({col_letter}2,10)=0,NOT(ISBLANK({col_letter}2)))',
                    'format':   green_highlight_format
                }
            )

        # Optional: Datumsformat für 'Geburtsdatum (TT.MM.JJJJ)'
        if 'Geburtsdatum (TT.MM.JJJJ)' in sheet_cols:
            geb_de_col_idx = sheet_cols.index('Geburtsdatum (TT.MM.JJJJ)')
            worksheet.set_column(geb_de_col_idx, geb_de_col_idx, 18, date_format_german)

        # EHRENMITGLIED- und weitere Rollenhervorhebung
        if mitgliedschaft_col and mitgliedschaft_col in sheet_cols:
            mg_col_idx = sheet_cols.index(mitgliedschaft_col)
            mg_col_letter = chr(ord('A') + mg_col_idx)
            row_end = len(export_df)
            # Formeln für die Hervorhebung Ehrenmitglied etc.
            for rolle in ["Ehrenmitglied", "Ehrenpräsident", "Prinzenrolle"]:
                worksheet.conditional_format(
                    f'A2:{chr(ord("A")+len(sheet_cols)-1)}{row_end+1}',
                    {
                        'type': 'formula',
                        'criteria': f'=ISNUMBER(SEARCH("{rolle}", ${mg_col_letter}2))',
                        'format': yellow_highlight_format
                    }
                )
    # Nach Monatsblättern: 'Ohne Geburtsdatum'-Sheet erstellen
    df_ohne_datum = df[df['Geburtsdatum'].isna()].copy()

    if not df_ohne_datum.empty:
        # Eventuell Mitgiedschaftsspalte wie oben suchen
        mitgliedschaft_col = None
        for mgl_col in ['Mitgliedschaft', 'mitgliedschaft', 'Mitglied', 'mitglied']:
            if mgl_col in df_ohne_datum.columns:
                mitgliedschaft_col = mgl_col
                break
        # Anzeigeformat-Spalte hinzufügen (wird leer bleiben)
        df_ohne_datum['Geburtsdatum (TT.MM.JJJJ)'] = ""
        # Gleiches Spaltenlayout wie bei den Monatsblättern
        desired_cols = [
            'Vorname', 'Nachname', 'Geburtsdatum (TT.MM.JJJJ)',
            'Firma', 'Strasse', 'PLZ', 'Ort', 'Alter 2026'
        ]
        if mitgliedschaft_col:
            desired_cols.append(mitgliedschaft_col)
        sheet_cols = [c for c in desired_cols if c in df_ohne_datum.columns]
        export_df_ohne = df_ohne_datum[sheet_cols]
        # Schreibe in ein weiteres Sheet nach Dezember
        export_df_ohne.to_excel(writer, sheet_name='Ohne Geburtsdatum', index=False)

        # Zugriff aufs Sheet für Formatierungen
        workbook  = writer.book
        worksheet = writer.sheets['Ohne Geburtsdatum']
        # Fettdruck für Header
        bold_format = workbook.add_format({'bold': True})
        worksheet.set_row(0, None, bold_format)
        worksheet.set_column(0, len(sheet_cols)-1, 18)

# Hinweis-Output für den User
print("Geburtstagsliste_2026.xlsx erfolgreich erstellt: Pro Monat ein Tabellenblatt, runde Geburtstage grün markiert, Ehrenmitglieder/-präsidenten/Prinzenrolle gelb hervorgehoben. Die Spalte 'Geburtsdatum' wird nur noch als TT.MM.JJJJ angezeigt, danach folgen Firma, Strasse, PLZ, Ort.")
print("Zusätzlicher Reiter 'Ohne Geburtsdatum' für alle Einträge ohne Datum hinzugefügt.")


Geburtstagsliste_2026.xlsx erfolgreich erstellt: Pro Monat ein Tabellenblatt, runde Geburtstage grün markiert, Ehrenmitglieder/-präsidenten/Prinzenrolle gelb hervorgehoben. Die Spalte 'Geburtsdatum' wird nur noch als TT.MM.JJJJ angezeigt, danach folgen Firma, Strasse, PLZ, Ort.


In [22]:
# Anzahl ausgegebener Namen, Geburtsdaten und fehlgeschlagene (fehlende) Geburtsdaten ausgeben
anzahl_namen = export_df['Vorname'].notna().sum()
anzahl_geburtsdaten = export_df['Geburtsdatum (TT.MM.JJJJ)'].notna().sum() if 'Geburtsdatum (TT.MM.JJJJ)' in export_df.columns else 0
anzahl_fehlende_geburtsdaten = export_df['Geburtsdatum (TT.MM.JJJJ)'].isna().sum() if 'Geburtsdatum (TT.MM.JJJJ)' in export_df.columns else len(export_df)

print(f"Anzahl Namen in der Liste: {anzahl_namen}")
print(f"Anzahl Geburtsdaten in der Liste: {anzahl_geburtsdaten}")
print(f"Fehlgeschlagene (fehlende) Geburtsdaten: {anzahl_fehlende_geburtsdaten}")


Anzahl Namen in der Liste: 33
Anzahl Geburtsdaten in der Liste: 33
Fehlgeschlagene (fehlende) Geburtsdaten: 0
